In [48]:
import helpers
import numpy as np
import re
import math
from tqdm import tqdm
import copy
import networkx as nx
from collections import Counter
import ast
import itertools as it

data = """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]"""

with open("input_18.txt", 'r') as f:
    data = f.read()
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = re.findall("(\d+),(\d+) -> (\d+),(\d+)", data)
#data = [x.rstrip() for x in data.split("\n\n") if x]


data = data.lstrip()
data = data.rstrip()
#data = np.array([np.array([int(y) for y in x]) for x in data.split("\n")])

#data = [x.split("-") for x in data.split("\n")]
#data_hex = "9C0141080250320F1802104A08"

#data_l = ast.literal_eval(data)
#data_l
data = data.split("\n")
print(data)

['[[[[7,2],[0,2]],9],[[[7,8],6],[0,[2,3]]]]', '[[[0,9],3],1]', '[[[[0,5],6],[[0,6],[7,8]]],[[[2,2],[1,5]],[9,5]]]', '[[2,1],[[3,1],[[3,2],9]]]', '[[[[9,3],[7,5]],[5,9]],[[0,[0,4]],2]]', '[[[9,9],4],[8,[[4,9],7]]]', '[[[1,9],[[8,3],[6,1]]],[5,1]]', '[[[[8,6],[1,3]],[3,[1,1]]],[[[6,4],[0,4]],[[0,0],3]]]', '[[[[6,4],[3,3]],[7,5]],[[6,5],[0,4]]]', '[[[[6,4],[2,3]],5],[9,[[4,4],[7,7]]]]', '[[1,[[7,9],1]],[[[8,2],5],[[7,2],8]]]', '[[[2,[7,2]],[5,3]],[1,5]]', '[[[[2,2],[1,5]],[[1,6],[0,5]]],[6,[9,[9,9]]]]', '[[7,2],[[3,8],[5,[6,7]]]]', '[[[6,5],[[8,0],1]],[2,[6,0]]]', '[[[8,7],6],[[[7,2],[7,0]],3]]', '[[[[9,1],[0,1]],[0,4]],[8,0]]', '[[[8,2],[8,[2,7]]],[[2,[2,6]],[2,[1,2]]]]', '[[[1,7],[[0,0],3]],[[3,[3,6]],[6,3]]]', '[[[[5,9],[1,2]],[0,0]],[[8,8],9]]', '[[[1,2],[4,2]],[3,4]]', '[[[9,[0,0]],[[3,5],7]],[[[8,0],[2,1]],[3,[1,5]]]]', '[[[[9,2],[7,1]],[[0,4],5]],[9,2]]', '[[3,[6,1]],2]', '[[[0,[0,6]],[4,[5,9]]],[[5,9],1]]', '[[[9,[8,3]],7],[[0,7],[[8,0],[9,2]]]]', '[[[[0,7],7],[[1,8],[4,4]]],[[0,[

In [49]:
def magnitude(l):
    if type(l) is list: 
        return  3 * magnitude(l[0]) + 2 * magnitude(l[1])
    else:
        return l
    
def add(d1, d2):
    if not d1 and d2:
        return d2
    d = str([ast.literal_eval(d1),ast.literal_eval(d2)]).replace(" ", "")
    #print(d)
    while True:
        #print(d)
        while not d == explode(d):
            #print(f"before explode: {d}")
            d = explode(d)
            #print(f"exploded: {d}")
        if not d == split(d):
            #print(f"before split: {d}")
            d = split(d)
            #print(f"split: {d}")
            continue
        break
    return d

def explode(d):
    depth = 0
    for i, x in enumerate(d):
        noFound = False
        if x == "[":
            depth += 1
            if depth == 5:
                start_pos = i
                pair = ""
        if depth == 5:
            pair += x
        if x == "]":
            if depth == 5:
                pair = ast.literal_eval(pair)
                end_pos = i
                break
            depth -= 1
        noFound = True
    if noFound:
        return d
    #print((start_pos, end_pos), pair)
    d = d[:start_pos] + "0" + d[end_pos+1:]
    #print(d)
    #print(d[start_pos-2], d[start_pos])
    
    right_num = ""
    finish = False
    for x in range(start_pos+1, len(d)):
        if d[x].isdigit():
            if not finish:
                right_start = x
            right_num += d[x]
            finish = True
        elif finish:
            right_end = x
            break
    if finish:
        right_num = int(right_num)
        #print(right_num)
        d = d[:right_start] + str(right_num + pair[1]) + d[right_end:] 
    #print(d)
    
    left_num = ""
    finish = False
    #print(d)
    for x in range(start_pos-1, -1, -1):
        #print(x, d[x])
        if d[x].isdigit():
            if not finish:
                left_end = x
            left_num += d[x]
            finish = True
        elif finish:
            left_start = x
            break
    if finish:
        left_num = int(left_num[::-1])
        d = d[:left_start+1] + str(left_num + pair[0]) + d[left_end+1:] 
    return d

def split(d):
    val = re.search("[\d]{2,}", d)
    if not val:
        return d
    val = int(val.group())
    val_l = [math.floor(val/2),math.ceil(val/2)]
    val = re.sub("[\d]{2,}", str(val_l).replace(" ", ""), d, 1)
    return val
            

In [50]:
assert explode("[[[[[9,8],1],2],3],4]") == "[[[[0,9],2],3],4]"
assert explode("[7,[6,[5,[4,[3,2]]]]]") == "[7,[6,[5,[7,0]]]]"
assert explode("[[6,[5,[4,[3,2]]]],1]") == "[[6,[5,[7,0]]],3]"
assert explode("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]") == "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"
assert explode("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]") == "[[3,[2,[8,0]]],[9,[5,[7,0]]]]"
assert add("[[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]]","[2,9]") == "[[[[6,6],[7,7]],[[0,7],[7,7]]],[[[5,5],[5,6]],9]]"

In [51]:
data = it.permutations(data, 2)
largest_mag = 0
for p1, p2 in data:
    p = add(p1,p2)
    #print(p, magnitude(ast.literal_eval(p)))
    largest_mag = max(magnitude(ast.literal_eval(p)), largest_mag)
print(largest_mag)


4559


In [4]:
ans = ""
for x in data.split("\n"):
    ans = add(ans,x)
    #print(ans)
print(ans)

print(magnitude(ast.literal_eval(ans)))

[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]
4140


In [192]:
p = "[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]"

while True:
    while not p == explode(p):
        p = explode(p)
    if not p == split(p):
        p = split(p)
        continue
    break
print(p)

[[[[0,7],4],[[7,8],[6,0]]],[8,1]]


In [6]:
data

['[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
 '[[[5,[2,8]],4],[5,[[9,9],0]]]',
 '[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]',
 '[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]',
 '[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]',
 '[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]',
 '[[[[5,4],[7,7]],8],[[8,3],8]]',
 '[[9,3],[[9,9],[6,[4,9]]]]',
 '[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]',
 '[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]']

In [9]:
data = it.permutations(data, 2)

In [11]:
list(data)

[('[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
  '[[[5,[2,8]],4],[5,[[9,9],0]]]'),
 ('[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
  '[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]'),
 ('[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
  '[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]'),
 ('[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
  '[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]'),
 ('[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
  '[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]'),
 ('[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
  '[[[[5,4],[7,7]],8],[[8,3],8]]'),
 ('[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
  '[[9,3],[[9,9],[6,[4,9]]]]'),
 ('[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
  '[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]'),
 ('[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
  '[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]'),
 ('[[[5,[2,8]],4],[5,[[9,9],0]]]',
  '[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]'),
 ('[[[5,[2,8]],4],[5,[[9,9],0]]]', '[6,[[[6,2],[5,

In [28]:
magnitude(ast.literal_eval(add("[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]", "[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]")))

3993